<a href="https://colab.research.google.com/github/idiig/lah-c655-dh-rag-demo-2025/blob/master/LAH_C655_DH_RAG_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
---
title: LAH.C655 Building a RAG for querying humanities knowledge
author:
  - name: Xudong Chen
    corresponding: false
    roles: []
    id: jc
    orcid: 0000-0002-4542-2878
    email: xchen@shs.ens.titech.ac.jp
    affiliation:
      - name: Tokyo Institute of Technology
        city: Tokyo
        country: Japan
        url: 'https://www.titech.ac.jp/english/'
        isni: 0000000121792105
        ror: 0112mx960
  - name: Hilofumi Yamamoto
    corresponding: true
    roles: []
    id: jc
    orcid: 0000-0001-6876-139X
    email: yamagen@lia.titech.ac.jp
    affiliation:
      - name: Tokyo Institute of Technology
        city: Tokyo
        country: Japan
        url: 'https://www.titech.ac.jp/english/'
        isni: 0000000121792105
        ror: 0112mx960
date: 2024/10/10
abstract: |
  A Retrieval-Augmented Generation (RAG) system for querying humanities knowledge.
keywords:
  - RAG
  - Semiotics
  - Digital Humanities
license: CC BY
copyright:
  holder: 'Xudong Chen, Hilofumi Yamamoto'
  year: 2024
funding: ''
format:
  html:
    code-links:
      - text: Data Import Code
        icon: file-code
        href: 'https://gist.github.com/idiig/d62a55511c9d3e49e9a0f64945f490b4'
    theme: default
    toc: true
    toc-title: Table of Contents
    toc-location: right-body
    number-sections: true
    html-math-method: katex
    fig_caption: true
    cap-location: margin
    reference-location: margin
    citation-location: document
    code-fold: false
    fig-format: svg
    fig-width: 6
    fig-height: 3.71
execute:
  echo: false
  freeze: auto
  message: false
crossref:
  fig-title: Figure
  tbl-title: Table
  title-delim: ':\quad'
  fig-prefix: Figure
  tbl-prefix: Table
  sec-prefix: Section
  eq-prefix: Eq.
jupyter:
  kernel: "python3"  # Specify the kernel for the notebook
---

# LAH.C655 Building a RAG for querying humanities knowledge

---
この授業では、**人文学系のデータ**を用いて **RAGシステム** を構築します。

内容：

1. まず、RAGの基本を学びます。  
2. 次に、人文学分野に関連するリソースを探ります。  
3. その後、各自が作りたいRAGシステムのタイプについて議論します。  
4. 最後に、READMEファイルを作成し、GitHubリポジトリに更新します。  

In this class, we will be building a **RAG system** using **humanities data**.

Here’s what we’ll cover:

1. We’ll start with the basics of RAG.
2. Then, we’ll explore resources related to the humanities field.
3. After that, we’ll discuss the type of RAG system you'd like to create.
4. Finally, we’ll write a README file and update it to our GitHub repository.

## Retrieval-Augmented Generation (RAG)

---
**Retrieval-Augmented Generation (RAG)** とは、情報検索と生成モデルを組み合わせた手法です。  
応答を生成する際、まず **外部知識ベース**（文書やデータベースなど）から関連情報を検索し、その取得情報をもとに応答を生成します。  
この方法により、生成モデルはより多くの情報源を活用し、**正確で文脈に即した回答**を出力できるようになります。

準備すべきもの

1. **データの準備**

- **知識ベースの構築（ベクトルデータベース）**  
  検索のために知識ベースが必要です。これは文書群、データベース、または他の構造化情報源でも構いません。  
  文書は BERT や Sentence Transformers などの埋め込みモデルを用いてベクトル化し、FAISS や Milvus などのベクトルデータベースに保存します。

- **リトリーバ（Retriever）**  
  知識ベースから最も関連性の高い内容を抽出するための効率的な検索器が必要です。  
  近似最近傍探索（Approximate Nearest Neighbor, ANN）アルゴリズム（例：FAISS）を用いることで検索処理を実装できます。

2. **モデルの選定**

- **検索モデル（Retrieval Model）**  
  事前学習済みの検索モデル（例：Dense Passage Retrieval, DPR）や BERT 系モデルを利用できます。

- **生成モデル（Generative Model）**  
  GPT-3 や T5 などの事前学習済み生成モデルを用いてテキストを生成します。  
  実際には、検索モデルで候補文書や段落を取得し、それを生成モデルに入力して最終的な回答を得ます。

3. **実装の流れ**

- **ユーザの入力（クエリ）**  
   ユーザが質問や指示を入力します。  
- **リトリーバが関連文書を取得**  
   リトリーバが知識ベースから関連性の高い上位 K 件の文書を取得します。  
- **生成モデルが応答を生成**  
   取得された文書とユーザのクエリを組み合わせ、生成モデルに入力して最終的な応答を生成します。


Begin by introducing what Retrieval-Augmented Generation (RAG) is:
- **Retrieval-Augmented Generation (RAG)** is a technique that combines information retrieval with generative models. When generating responses, it first retrieves relevant content from an **external knowledge base** (such as documents or databases) and then uses that retrieved information to generate a response. This approach enables generative models to provide **more accurate and contextually relevant answers** by leveraging a larger information source.

What we need to prepare:

  1. Prepare for **Data**
      - **Knowledge Base Construction (a vector database)**: A knowledge base is needed for retrieval. This can be a collection of documents, a database, or other structured information sources. The documents can be vectorized using embeddings (such as BERT or Sentence Transformers) and stored in a vector database (like FAISS or Milvus).
      - **Retriever**: An efficient retriever is required to extract the most relevant content from the knowledge base. You can use Approximate Nearest Neighbor (ANN) search algorithms, such as FAISS, to implement the retrieval process.

  2. **Model** selection
       - **Retrieval Model**: You can use a pre-trained retrieval model (e.g., Dense Passage Retrieval, DPR) or a BERT-based model for information retrieval.
       - **Generative Model**: A pre-trained generative model (like GPT-3 or T5) can be used to generate the text. In practice, the retrieval model is used to retrieve candidate documents or passages, which are then input into the generative model to produce the final answer.

  3. **Implementation** workflow
      
      - **User Input Query**: The user inputs a query (a question or command).
      - **Retriever Fetches Relevant Documents**: The retriever fetches the top K relevant documents from the knowledge base.
      - **Generative Model Produces the Response**: The retrieved documents are combined with the user query and input into the generative model to generate the final response.

## In this Example Demo

---
デモで使用するデータとツール

1. **データ**

- **ソース**  
  [Semiotics for Beginners](http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/)  
  記号論（[Semiotics](https://en.wikipedia.org/wiki/Semiotics)）の優れた教科書であり、  
  [Daniel Chandler](https://en.wikipedia.org/wiki/Daniel_Chandler) によって執筆されています。  
  このHTML版をソース文書として利用します。

- **埋め込み方法（Embedding method）**  
  [OpenAIEmbeddings](https://platform.openai.com/docs/guides/embeddings) のデフォルトモデルを使用し、文書をベクトルに変換します。

- **ベクトルデータベース（Vector DB）**  
  ベクトルの保存には軽量でオープンソースの埋め込みデータベース [Chroma](https://www.trychroma.com/) を使用します。

- **検索方法（Retrieval method）**  
  検索には Chroma のデフォルトリトリーバを使用します。今回の目的には十分です。

2. **モデル**

- **リトリーバモデル（Retriever model）**  
  検索には OpenAIEmbedding のデフォルトモデルを使用します。

- **生成モデル（Generative model）**  
  生成には OpenAI の `gpt-4o-mini` モデルを使用します。

3. **実装**

- [RAGシステム](https://en.wikipedia.org/wiki/Retrieval-augmented_generation) の構築には [LangChain](https://ja.wikipedia.org/wiki/LangChain) を利用します。  
LangChain は、大規模言語モデル（LLM）を活用したアプリケーションを構築するための強力なフレームワークであり、  
モデルと外部ツール・データベースをモジュール的に組み合わせることを可能にします。

Here’s the data and tools we’ll be using for our RAG demo:

1. **Data**
   - **Source**: [Semiotics for Beginners](http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/), which is a great textbook for [Semiotics](https://en.wikipedia.org/wiki/Semiotics) (記号論), written by [Daniel Chandler](https://en.wikipedia.org/wiki/Daniel_Chandler). We’ll use the HTML version as our source document.
   - **Embedding method**: We’ll use [OpenAIEmbeddings](https://platform.openai.com/docs/guides/embeddings) with the default model to convert our documents into vectors.
   - **Vector DB method**: For storing these vectors, we’ll go with [Chroma](https://www.trychroma.com/), a lightweight, open-source embedding database.
   - **Retrieval method**: Chroma’s default retrieval method will work just fine for what we need.

2. **Model**
   - **Retriever model**: We’ll stick with the OpenAIEmbedding default model for retrieval.
   - **Generative model**: Again, we’ll use the OpenAI's `gpt-4o-mini` model for generation.

3. **Implementation**: We’ll be building all of this using [LangChain](https://ja.wikipedia.org/wiki/LangChain).
    - LangChain is a powerful framework for building applications powered by language models (LLMs), such as Retrieval-Augmented Generation (RAG) systems. It enables developers to combine models with external tools and databases in a modular way.


## Find humanities resources that you are interested in

---
人文学に関連するリソースを探してみましょう。以下はいくつかの例です。

**哲学:**

- [Stanford Encyclopedia of Philosophy Archive](https://plato.stanford.edu/archIves/fall2024/index.html)

**日本文学・国語学・言語学:**

- [日文研公開データベース](https://www.nichibun.ac.jp/ja/db/)
- [青空文庫](https://www.aozora.gr.jp/)
- [雑誌『国語学』全文データベース](https://bibdb.ninjal.ac.jp/SJL/list.html)

**言語学:**

- [Database of Cross-Linguistic Colexifications](https://clics.clld.org/download)
- [The World Atlas of Language Structures (WALS)](https://wals.info/)


Some examples:

Philosophy:

- [Stanford Encyclopedia of Philosophy Archive](https://plato.stanford.edu/archIves/fall2024/index.html)

Japanese literature, philology and linguistics:

- [日文研公開データベース](https://www.nichibun.ac.jp/ja/db/)
- [青空文庫](https://www.aozora.gr.jp/)
- [雑誌『国語学』全文データベース](https://bibdb.ninjal.ac.jp/SJL/list.html)

Linguistics:

- [Database of Cross-Linguistic Colexifications](https://clics.clld.org/download)
- [The World Atlas of Language Structures (WALS)](https://wals.info/)



# 0 | Required packages and settings

---
RAGシステムを構築するために、以下のツールをインストールします。

1. **`langchain`**  
   言語モデルを用いたアプリケーションを構築するための主要パッケージです。  
   データ処理、情報検索、応答生成の流れを整理して構築することができます。

2. **`langchain-community`**  
   LangChain コミュニティによる拡張ツールや追加機能を含むパッケージです。  
   これらを利用することで、LangChain の機能をさらに強化できます。

3. **`langchain_text_splitters`**  
   大きな文書を小さなチャンクに分割するためのツールです。  
   RAGシステムでは、長文を処理しやすくするために分割して利用します。

4. **`langchain_chroma`**  
   LangChain と **Chroma**（ベクトルデータベース）を接続するためのパッケージです。  
   質問に答える際、最も関連性の高い情報を検索・取得する中核的な役割を担います。

これらをインストールすることで、RAGシステムを段階的に構築するための準備が整います。

インストールコマンド：

Here are the common tools we will need to install for our RAG system:

1. **`langchain`**: This is the main package that contains everything we need to build applications using language models. It helps us organize how we process data, retrieve information, and generate responses.

2. **`langchain-community`**: This package includes extra tools and features created by the LangChain community. These community contributions add new ways to use LangChain and can make our system even better.

3. **`langchain_text_splitters`**: This package helps us break large documents into smaller chunks. In our RAG system, we’ll split long texts into pieces so that they are easier to process when generating responses.

4. **`langchain_chroma`**: We’ll use this to connect LangChain with **Chroma**, a tool that stores and manages our data as vectors. This is an important part of retrieving the most relevant information when answering questions.

By installing these, we’ll have all the necessary tools to build our system step by step!

To install them, use the following command:

In [1]:
!pip install -qU langchain langchain-community langchain_text_splitters langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.4 MB/s eta 0

In [2]:
# Additional requirement to use LLM from OpenAI
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.2 MB/s eta 0:00:00


In [ ]:
# Additional requirement to use open-source LLM from hugging face community
!pip install -q torch transformers transformers accelerate sentence-transformers faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.8 MB/s eta 0:00:00



---

Google Colab 上で実行する場合、LangChain を正しくインストールするために UTF-8 ロケールを有効にしておく必要があります。  

If running in Google Colab, you may need to run this cell to make sure you're using UTF-8 locale to install LangChain


In [3]:
# If running in Google Colab, you may need to run this cell to make sure you're using UTF-8 locale to install LangChain
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# 1 | Large Language Model (LLM) selection

---

このデモでは、[LangChain](https://ja.wikipedia.org/wiki/LangChain) を用いて **大規模言語モデル（Large Language Model, LLM）** を利用します。  
今回のデモでは [OpenAI](https://openai.com/) のモデルを使用しますが、
以下のような他の選択肢も利用可能です。

- Anthropic  
- Azure  
- Google  
- Cohere  
- NVIDIA  
- FireworksAI  
- Groq  

これらのモデルを使用する場合、それぞれのサービスで **APIキー** を取得する必要があります。つまり、各プロバイダにアカウント登録を行い、アクセス用のAPIキーを発行する必要があります。  

**なお、これらのサービスは無料ではありません。**

異なるモデルのセットアップ手順については、公式の[LangChainドキュメント](https://python.langchain.com/docs/tutorials/rag/)に詳しいスクリプトと説明が掲載されています。

有料サービスに加えて、無料で利用できるオープンソースのLLMもあります。たとえば、**Hugging Face Zephyr** を試すことができます。詳細は以下の [cookbook](https://github.com/huggingface/cookbook/blob/main/notebooks/en/rag_zephyr_langchain.ipynb) を参照してください。

さらに幅広いオープンソースLLMの一覧は、[Open-source LLM leaderboard](https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard)で確認できます。


In this demo, we are using a **Large Language Model (LLM)** with [LangChain](https://ja.wikipedia.org/wiki/LangChain). For this demo, we will be utilizing the LLM from [OpenAI](https://openai.com/), but there are several other alternatives you can explore, including:

- Anthropic
- Azure
- Google
- Cohere
- NVIDIA
- FireworksAI
- Groq

If you choose to use one of these models, you will need to obtain the corresponding API keys. This means you’ll need to sign up for an account with the provider and generate API keys for access. **Please note that these services are not free**.

The procedures for setting up different models can be found in the official [LangChain documentation](https://python.langchain.com/docs/tutorials/rag/), which provides setup scripts for various models.

In addition to paid services, there are also free, open-source LLMs you can use. For instance, you could try **Hugging Face Zephyr**. You can find more information in this helpful [cookbook](https://github.com/huggingface/cookbook/blob/main/notebooks/en/rag_zephyr_langchain.ipynb).

For a broader range of open-source LLM options, take a look at the [Open-source LLM leaderboard](https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard).


## 1.1 | Non-free LLM, e.g., OpenAI gpt-4o

---

有料の LLM（たとえば **OpenAI の GPT-4o**）を使用して、埋め込み・検索・テキスト生成などのタスクを実行します。以下の手順で、APIキーを安全に入力し、モデルを初期化します。

We need to set up and use a non-free LLM, like **OpenAI's GPT-4o**, for tasks such as embedding, retrieval, and text generation. We will securely input our API key and initialize the model.

In [4]:
# Non-free LLM, e.g., OpenAI gpt-4o
# you need to input the api key
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")  # for embedding, retriever and generator

··········


## 1.2 | Alternative: Free LLM, e.g., Hugging Face Zephyr

---
無料の代替案をご希望の場合は、**Hugging Face Zephyr**のようなモデルを使用できます。これはHugging Faceを通じて利用可能なオープンソースのLLMで、多くのプロジェクトにとってコスト効率の良い選択肢となります。

ただし、**ご注意ください**：RAMの制限により、このオプションは**Google Colab**での実行には適していない可能性があります。

If you'd prefer a free alternative, you can use models like **Hugging Face Zephyr**. It is an open-source LLM available through Hugging Face, making it a cost-effective choice for many projects.

However, **please note** that due to RAM limitations, this option may not be suitable for running on **Google Colab**.

In [ ]:
# DO NOT RUN in Colab
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "HuggingFaceH4/zephyr-7b-beta"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# DO NOT RUN in Colab
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 2 | (Humanities Domain-specific) Documents / Data source selection

---

このプロジェクトでは、**人文学**に関連するドメイン固有の文書を使用してRAGシステムを構築します。例として、記号論の入門教科書である[Semiotics for Beginners](http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/)を使用しました。

この特定のソースはHTML文書ですが、LangChainは以下を含む様々な文書タイプをサポートしています：

LangChainはHTML以外にも様々な文書タイプをサポートしています：

- **PDF**：PDFLoaderを使用してPDFファイルからテキストを抽出します。
- **TXT**：プレーンテキストファイルを直接処理します。
- **HTML**：HTML文書やウェブページを解析します。
- **DOC/DOCX**：`python-docx`のようなPythonライブラリを使用してMicrosoft Wordファイルを扱います。
- **EPUB**：EPUB解析ライブラリを使用して電子書籍を扱います。
- **CSV**：CSVファイルを使用してテーブルからデータを抽出します。

プロジェクトのニーズに基づいて、これらのフォーマットを自由に探索してください。詳しいガイダンスについては、LangChainのドキュメントを参照するか、関連するPythonライブラリを検索してください。

これらのフォーマットやニーズに合った他のフォーマットを自由に使用してください。**ドキュメントローダー**については https://python.langchain.com/docs/integrations/document_loaders/ を参照してください。

**重要な注意事項**：
外部リソースを扱う際は、選択した文書のライセンス条項を必ず確認してください。適切な許可を得ずに、著作権で保護されたリソースをリポジトリに直接含めることはできません。使用を予定している外部リソースについては、必ずライセンスや許可を確認してください。

In this project, we will be using domain-specific documents related to the **Humanities** to build our RAG system. As an example, I’ve used [Semiotics for Beginners](http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/), which is an introductory textbook on semiotics (記号論).

This particular source is an HTML document, but LangChain supports various document types, including:

LangChain supports various document types beyond HTML, including:

- **PDF**: Use PDFLoader to extract text from PDF files.
- **TXT**: Process plain text files directly.
- **HTML**: Parse HTML documents or web pages.
- **DOC/DOCX**: Use Python libraries like `python-docx` to handle Microsoft Word files.
- **EPUB**: Use EPUB parsing libraries to work with e-books.
- **CSV**: Extract data from tables using CSV files.

Feel free to explore these formats based on your project needs. You can refer to LangChain's documentation or search for relevant Python libraries for further guidance.

Feel free to use any of these formats or others that match your needs. For **Document Loader** see https://python.langchain.com/docs/integrations/document_loaders/.

**Important Note**:
Since we are working with external resources, make sure you check the licensing terms of the documents you choose. We cannot include copyrighted resources directly in our repository without obtaining proper permission. Always confirm the license or permissions for any external resources you plan to use.


###  2.1 | Source: [Semiotics for Beginners](http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/)



---
まず、`requests`ライブラリを使用してウェブページのコンテンツを取得し、`bs4`の`BeautifulSoup`を使用してHTMLを解析します。次に、スクリプトはすべてのリンク（`href`属性を持つ`<a>`タグ）を抽出して処理し、アンカーや`mailto:`リンクなどの不要なリンクを除外し、相対URLを絶対URLに変換します。

We firstly use the `requests` library to fetch the content of a webpage, and `BeautifulSoup` from `bs4` to parse the HTML. The script then extracts all the links (`<a>` tags with `href` attributes) and processes them, filtering out unwanted links like anchors or `mailto:` links, and converting relative URLs into absolute ones.


In [5]:
import requests
import bs4

# Fetch the webpage content
url = "http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/"
response = requests.get(url)
html_content = response.text

# Use BeautifulSoup to parse the HTML content
soup = bs4.BeautifulSoup(html_content, "html.parser")

# Extract all <a> tags with href attributes
links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    # Filter out unwanted anchors or mailto links
    if not href.startswith("#") and not href.startswith("mailto:"):
        # If it's a relative link, convert it to an absolute link
        if not href.startswith("http"):
            href = url + href
        links.append(href)

In [6]:
links

['http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem0a_japanese.html',
 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem01_japanese.html',
 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem08_japanese.html',
 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem02_japanese.html',
 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem08b_japanese.html',
 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem02a_japanese.html',
 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem08c_japanese.html',
 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem03_japanese.html',
 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem08a_japanese.html',
 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem04_japanese.html',
 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem09_japanese.html',
 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanes

---
次に、上記のリンクから関連するテキストコンテンツを読み込みます。特に段落レベルの情報に焦点を当てます。これにより、取得したコンテンツがより構造化され、検索拡張生成（RAG）プロセスの後の段階で使いやすくなります。

Then we load relevant textual content from the above links, specifically focusing on paragraph-level information. This makes the retrieved content more structured and usable for later stages in the Retrieval-Augmented Generation (RAG) process.


In [7]:
from langchain_community.document_loaders import WebBaseLoader

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=tuple(links),  # Provide the list of URLs as a tuple
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer("p")  # Parse only <p> tags for content extraction
    ),
)

docs = loader.load()

In [8]:
docs

[Document(metadata={'source': 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem0a_japanese.html'}, page_content='\n\u3000どうして、そして誰のために "Semiotics for Beginners" を書くようになったか、たびたび訊ねられます。最初、１９９４年に私自身と University of Wales, Aberystwyth のメディア教育コースの学部３年生のために書きました。個人的な意見ですが、マスメディアを学ぶ人、またはコミュニケーションや文化研究にとって記号論は必須だと思っています。その時点では、良いテキストが無かったので、自分自身の目的と学生の目的にあったテキストを作成することにしました。それは、記号論に関する理解を進め、明確にするための方法でした。多くの読者と同様に、記号論の勉強は理解できないような本によって挫折させられてきました。教育者としては、そのような本の著者はまったく恥じ入るべきだと感じています。意味の生成 (meaning-making) という主題は、広範な読者層にとって理解可能で魅力的なものでありますが、大部分の本は、それを混乱した、退屈でかつ非常に不明瞭なものにしようとしているように思えます。\n\n\u3000大学において、このテキストを書くことを優先させたのは、それが展開していくことが期待できたためですが、最初のテキストを書いてから、その領域は大きく拡大し、マスメディアのみならず文学、美術や神話などにもしばしば言及するようになっています。記号が私を魅了するのは、学術領域の ‘境界’を横断し、また明らかに異なる現象を関連付ける楽しみを支援してくれるからでありました。私はテキストとともに成長しました：それはたやすく改定できるオンライン形式なので、私がそれよりも ‘成長してしまった’という心配はせずにすんでいます。一方、博学でないので取り上げられなかった項目も、必然的に多くなっています。このテキストでは、内容を人間の記号過程に限定しました。このため、動物の行動やコミュニケーション（動物記号論）のような記号論の他の流れを紹介していません。また、コンピュータ間のコミュニケーションにも触

## 2.2 | Data preprocess

---
ここでは、LangChainの`RecursiveCharacterTextSplitter`を使用して、大きな文書をより小さく管理しやすいチャンクに分割します。これは、小さな部分に分けて処理またはインデックス化する必要がある長いテキストを扱う際に特に便利です。

Here, we use the `RecursiveCharacterTextSplitter` from LangChain to split large documents into smaller, manageable chunks. This is especially useful when working with long texts that need to be processed or indexed in smaller parts.


In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
splits = text_splitter.split_documents(docs)

In [10]:
splits

[Document(metadata={'source': 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem0a_japanese.html'}, page_content='どうして、そして誰のために "Semiotics for Beginners" を書くようになったか、たびたび訊ねられます。最初、１９９４年に私自身と University of Wales, Aberystwyth のメディア教育コースの学部３年生のために書きました。個人的な意見ですが、マスメディアを学ぶ人、またはコミュニケーションや文化研究にとって記号論は必須だと思っています。その時点では、良いテキストが無かったので、自分自身の目的と学生の目的にあったテキストを作成することにしました。それは、記号論に関する理解を進め、明確にするための方法でした。多くの読者と同様に、記号論の勉強は理解できないような本によって挫折させられてきました。教育者としては、そのような本の著者はまったく恥じ入るべきだと感じています。意味の生成 (meaning-making) という主題は、広範な読者層にとって理解可能で魅力的なものでありますが、大部分の本は、それを混乱した、退屈でかつ非常に不明瞭なものにしようとしているように思えます。'),
 Document(metadata={'source': 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem0a_japanese.html'}, page_content='大学において、このテキストを書くことを優先させたのは、それが展開していくことが期待できたためですが、最初のテキストを書いてから、その領域は大きく拡大し、マスメディアのみならず文学、美術や神話などにもしばしば言及するようになっています。記号が私を魅了するのは、学術領域の ‘境界’を横断し、また明らかに異なる現象を関連付ける楽しみを支援してくれるからでありました。私はテキストとともに成長しました：それはたやすく改定できるオンライン形式なので、私がそれよりも'),
 Document(metadata={'source': 'http://www.vi

# 3 | Vector Database (the knowledge base)

---
このセクションでは、非無料の埋め込みモデル（OpenAI）と、埋め込みを保存するための軽量なオープンソースのベクターデータベースである**Chroma**を使用してベクターデータベースを構築します。

In this section, we are building a vector database using OpenAI's non-free embedding model and **Chroma**, which is a lightweight, open-source vector database for storing embeddings.


## 3.1 | Embedding model

---
埋め込みモデル（Embedding Model）は、テキストデータを数値ベクトルに変換するための機械学習モデルです。

このモデルは、単語や文章などのテキストを高次元の数値ベクトル（通常は数百から数千次元）に変換します。これにより、コンピュータがテキストの意味を理解し、処理できるようになります。

**主な特徴：**
- **意味の保持**：似た意味を持つテキストは、ベクトル空間上で近い位置に配置されます
- **数学的演算が可能**：テキスト間の類似度計算や検索が高速に実行できます
- **RAGシステムでの役割**：ユーザーの質問と文書の内容を同じベクトル空間に変換し、関連性の高い情報を効率的に検索できるようにします

例えば、「犬」と「子犬」は意味的に近いため、ベクトル空間上でも近い位置に配置され、「車」のような無関係な単語からは離れた位置に配置されます。

An embedding model is a machine learning model that converts text data into numerical vectors.

This model transforms text such as words or sentences into high-dimensional numerical vectors (typically hundreds to thousands of dimensions). This enables computers to understand and process the meaning of text.

**Key Features:**
- **Semantic Preservation**: Texts with similar meanings are positioned close to each other in the vector space
- **Mathematical Operations**: Enables fast computation of similarity between texts and efficient search operations
- **Role in RAG Systems**: Converts both user queries and document content into the same vector space, allowing for efficient retrieval of highly relevant information

For example, "dog" and "puppy" are semantically similar, so they are positioned close to each other in the vector space, while unrelated words like "car" are positioned farther away.

In [11]:
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model="text-embedding-3-large")

## 3.2 | Database

### 3.2.1 | Creating a New Database

---
新しいベクターデータベース（DB）をゼロから作成する必要がある場合、このコードは非無料の埋め込みモデルによって作成された埋め込みを使用してChromaでデータベースを初期化し、保存します。また、このデータベースにアクセスするためのリトリーバーを設定することもできます。

If you need to create a new vector database (DB) from scratch, this code will initialize the database using **Chroma** and store the embeddings created by the non-free embedding model. You can also set up a retriever to access this database.



In [12]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
  documents=splits,
  embedding=embedding_function,
  persist_directory="./chroma_db",
)

### 3.1.2 | Alternative: Reusing an Existing Database

---
ベクターデータベースをゼロから再構築する代わりに、事前に準備された既存の**Chroma**データベースを使用できます。これにより、以前に作成されたデータベースから埋め込みを直接読み込むことで、時間とリソースを節約できます。

Instead of rebuilding the vector database from scratch, we can use an existing **Chroma** database that has been prepared beforehand. This allows us to save time and resources by directly loading the embeddings from a previously created database.


In [ ]:
# I download my existing Chroma DB from my google drive
!gdown --folder https://drive.google.com/drive/folders/1Y1QT3zWIc0oPGJbGdSxP_MWvhz1CAecL?usp=sharing

Retrieving folder contents
Retrieving folder 1-1GjnKu2NXTrfgPBXGJA1WAl3KFKMH0w 04226cce-6d14-486a-a1b1-b65ba036747b
Processing file 1-EPv0jHtH6Qe-NkqQk8iMgfIXJfH99vM data_level0.bin
Processing file 1-DMSdwMRCpOE5-mFRspjQg0oc4Cz9exX header.bin
Processing file 1-PldyhMMlV2ZEuzENso42JYG2esl1ahz length.bin
Processing file 1-Ce6LKotMq6f1iI_umj6QeVltwkfiPB1 link_lists.bin
Processing file 1-5A0qnc5kUR4oxiicWcBayv-xDhCAVSu chroma.sqlite3
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-EPv0jHtH6Qe-NkqQk8iMgfIXJfH99vM
To: /content/chroma_db/04226cce-6d14-486a-a1b1-b65ba036747b/data_level0.bin
100% 12.4M/12.4M [00:00<00:00, 53.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-DMSdwMRCpOE5-mFRspjQg0oc4Cz9exX
To: /content/chroma_db/04226cce-6d14-486a-a1b1-b65ba036747b/header.bin
100% 100/100 [00:00<00:00, 532kB/s]
Downloading...
From: https://drive.google.com/uc?id=1-PldyhMMlV2Z

In [ ]:
# Build vector DB from existing DB
vectorstore = Chroma(
    embedding_function=embedding_function,
    persist_directory="./chroma_db"  # vector DB path
)

# 4 | Retriever

---
LangChainでは、このデータベースにアクセスするためのリトリーバーを1行で設定できます。

You can set up a retriever to access this database in LangChain with one line.

In [13]:
# Retriever using vector DB
retriever = vectorstore.as_retriever()

## 4.1 | Alternative: Buiding DB using free embedding model and FAISS

OpenAIのような非無料のLLMやChromaの代わりに、**無料のLLM**と**FAISS**を使用してベクターデータベースを構築する代替アプローチもあります。

説明：

1. **無料のモデル**：
   - OpenAIのような有料サービスを使用する代わりに、Hugging Faceから入手可能な無料のモデルを使用して文書の埋め込みを生成できます。これらの無料モデルは、実験と開発のコストを削減できます。

2. **FAISS**：
   - **FAISS**（Facebook AI Similarity Search）は、密ベクトルの効率的な類似性検索とクラスタリングのために設計されたオープンソースライブラリです。ベクトル埋め込みの保存と取得において、Chromaの優れた代替手段となります。
   - FAISSは、大規模なデータセットを扱う際の速度とスケーラビリティで特に知られています。ただし、FAISSは主にメモリ上で動作するため、大量のリソースを必要とする可能性があり、RAMが制限されているGoogle Colabのようなプラットフォームでは制約となる場合があります。

3. **なぜFAISSを使用するのか？**：
   - FAISSは、ベクトル類似性検索のための高性能で無料のソリューションを提供します。有料APIの使用が選択肢にないプロジェクトの場合、FAISSは検索拡張生成（RAG）システムを構築するための実行可能な代替手段となります。
   - 大規模なデータセットを処理でき、ベクトル埋め込みの検索に最適化されているため、学術分野や企業アプリケーションの両方で人気の選択肢となっています。

**重要な注意事項**：
Google ColabのRAM制限により、このアプローチはColab環境では期待通りに動作しない可能性があります。

We also have an alternative approach to building a vector database using a **free embedding models** and **FAISS** instead of a non-free LLM like OpenAI and Chroma.

Explanation:

1. **Free embedding model**:
   - Instead of using a paid service like OpenAI, free models such as those available from Hugging Face can be used to generate embeddings for the documents. These free models can reduce the cost of experimentation and development.

2. **FAISS**:
   - **FAISS** (Facebook AI Similarity Search) is an open-source library designed for efficient similarity search and clustering of dense vectors. It is an excellent alternative to Chroma for storing and retrieving vector embeddings.
   - FAISS is particularly known for its speed and scalability when working with large datasets. However, because FAISS operates primarily in memory, it may require significant resources, which can be a limitation on platforms like Google Colab due to restricted RAM.

3. **Why Use FAISS?**:
   - FAISS offers a high-performance, free solution for vector similarity search. For projects where using paid APIs is not an option, FAISS provides a viable alternative for building a retrieval-augmented generation (RAG) system.
   - It can handle large datasets and is well-optimized for searching through vector embeddings, making it a popular choice for academic and enterprise applications alike.

**Important Note**:
Due to RAM limitations in Google Colab, this approach may not work as expected in the Colab environment.


In [ ]:
# DO NOT RUN in Colab
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

db = FAISS.from_documents(
    splits,
    HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')
)

# 5 | Prompt (question + context) template

---
*プロンプトとは何か？*

**プロンプト**とは、LLMに与えられる最初の入力または質問であり、モデルが応答すべきタスクやクエリを定義するものです。RAGシステムにおいて、プロンプトは通常以下を含みます：
1. **ユーザーのクエリ**：ユーザーから提供される主要な質問またはタスク
2. **コンテキストとして取得された文書**：ベクターデータベースからリトリーバーによって取得された追加の情報。これらはLLMがクエリのコンテキストを理解し、より正確な応答を生成するのに役立ちます。

*RAGにおけるプロンプトの動作*：

1. **ユーザークエリ**：
   - ユーザーが質問をするか、入力を提供します（メインプロンプト）。
   
2. **検索**：
   - リトリーバーがベクターデータベースで関連文書を検索し、クエリに最も類似した文書のセットを返します。
   
3. **拡張プロンプト**：
   - 取得された文書は元のユーザークエリと組み合わされて、**拡張プロンプト**を作成します。この拡張プロンプトは、ユーザーの入力と関連するコンテキストの両方をLLMに提供し、詳細で正確な応答を生成しやすくします。
   
4. **応答生成**：
   - LLMは拡張プロンプトを処理し、ユーザーのクエリと取得された文書からの追加コンテキストの両方に基づいた応答を生成します。

*What is a Prompt*?

A **prompt** is the initial input or question given to the LLM, which defines the task or query for the model to respond to. In a RAG system, the prompt often includes:
1. **The User’s Query**: This is the main question or task provided by the user.
2. **Retrieved Documents as Context**: These are additional pieces of information retrieved by the retriever from the vector database. They help the LLM understand the context of the query and generate more accurate responses.

*How Prompts Work in RAG*:

1. **User Query**:
   - The user asks a question or provides an input (the main prompt).
   
2. **Retrieval**:
   - The retriever searches the vector database for relevant documents and returns a set of documents that are most similar to the query.
   
3. **Augmented Prompt**:
   - The retrieved documents are then combined with the original user query to create an **augmented prompt**. This augmented prompt provides the LLM with both the user’s input and relevant context, making it easier to generate a detailed and accurate response.
   
4. **Response Generation**:
   - The LLM processes the augmented prompt and generates a response that is informed by both the user’s query and the additional context from the retrieved documents.



In [14]:
# PromptTemplate class is used to create a custom prompt template for the LLM
from langchain.prompts import PromptTemplate

# Prompt template (`<||>` denotes section; `{}` denotes placeholder):
# `<|system|>`: defines the system's role, provide general instructions to LLM
# `{context}`: a placeholder for the context retrieved from the knowledge base.
#   It's dynamically replaced with relevant information related to the user's query.
# `<|user|>`: represents the user's input, where the user asks a question.
# {question}: a placeholder which will be replaced with the user's actual question.
# `<|assistant|>`: This is where the language model (assistant) generates its response,
#   based on both the context and the question.
prompt_template = """
<|system|>
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
keep the answer as concise as possible.
Use markdown formatting when displaying code.
Emphasis should be used to terminologies.
Always say "thanks for asking!" at the end of the answer.

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

"""

#  Create the PromptTemplate Instance
prompt = PromptTemplate(
    input_variables=[
        "context",
        "question",
        "sources"
        ],
    template=prompt_template,
)

# Or using availible prompts
# from langchain import hub
# from langchain_chroma import Chroma

# prompt use
# prompt = hub.pull("rlm/rag-prompt")

# 6 | Optional: Output formatter

### 6.1 | Document (context) formatter

---
処理された文書を単一の文字列にフォーマットする関数を定義します。これにより、表示やさらなる処理が簡単になります。複数の文書チャンクの内容を出力または可視化したい場合に便利です。

We define a function to format the processed documents into a single string for easier display or further processing. This is helpful when you want to output or visualize the content of multiple document chunks.


In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

### 6.2 | Source information formatter

---
次に、取得された文書からソース（メタデータ）を抽出してフォーマットする関数を定義します。これは、ソース情報（URLや文書タイトルなど）をコンテキストの一部として言語モデル（LLM）に渡したい場合に特に便利です。

Next, we define a function to extract and format the sources (metadata) from the retrieved documents. This is particularly useful when you want to pass the source information (such as URLs or document titles) as part of the context to a language model (LLM).


In [16]:
# Define a custom function to format sources from retrieved documents
def format_sources(docs):
    """
    Extracts and formats the sources (metadata) from the retrieved documents.
    This function ensures that the source information is passed to the LLM as part of the context.
    """
    # Extract sources (e.g., URLs or document titles) from metadata
    sources = [doc.metadata.get('source', 'Unknown Source') for doc in docs]
    # Join the sources into a single string to pass to the LLM
    return "Sources:\n" + "\n".join(sources)

### 6.3 | Answer formatter

---
次のコードは、Jupyterノートブックや、IPythonの表示機能をサポートする環境内で、出力（回答）をきれいにフォーマットされたMarkdown形式で表示するために使用されます。

The following code is used to display the output (answer) in a neatly formatted **Markdown** format within a Jupyter notebook or any environment that supports IPython display functions.


In [17]:
# To  display the output (answer) in a neatly formatted Markdown format
# within a Jupyter notebook
from IPython.display import Markdown, display

def display_answer(answer):
    display(Markdown(answer))

# 7 | Overall workflow by combining the above components

```
+----------+     +-------------+     +-------------+     +---------+
| Question |---->| Retrieval   |---->|   Context   |---->| Answer  |
|          |     |  (search)   |     |(Docs merged)|     |(LLM Gen)|
+----------+     +-------------+     +-------------+     +---------+
                       ^
                       |
                 +----------------+
                 |    Database /  |
                 |   (documents)  |
                 +----------------+
```

---

このセクションでは、上記のコンポーネントを組み合わせて、検索拡張生成（RAG）システムを使用した完全なワークフローを構築する方法について説明します。

**RAGchain**は、LangChainフレームワーク内の専門的なコンポーネントで、検索と生成を1つのシームレスなワークフローで組み合わせるプロセスを効率化するよう設計されています。RAGchainがこのプロセスをより効率的にする方法は以下の通りです：
- **リトリーバーとジェネレーターの統合**：RAGchainは、リトリーバー（関連文書を見つける）とジェネレーター（取得された文書に基づいて最終的な応答を作成する）の両方を1つの統合されたワークフローに組み合わせます。
- **コンテキストを考慮した生成**：検索を生成プロセスに直接統合することで、RAGchainは生成される応答がより正確で文脈的に関連性の高いものになることを保証します。
- **簡略化されたワークフロー**：RAGchainを使用すると、検索と生成の個別のステップを手動で処理する必要がありません。フレームワークがプロセス全体を処理するため、実装と保守がより簡単になります。

In this section, we discuss how to combine the above components to build a complete workflow using a Retrieval-Augmented Generation (RAG) system.

**RAGchain** is a specialized component within the LangChain framework designed to streamline the process of combining retrieval and generation in one seamless workflow. Here’s how RAGchain makes this process more efficient:
- **Integrates Retriever and Generator**: RAGchain combines both the retriever (which finds relevant documents) and the generator (which creates the final response based on the retrieved documents) into one unified workflow.
- **Context-aware Generation**: By integrating retrieval directly into the generation process, RAGchain ensures that the generated responses are more accurate and contextually relevant.
- **Simplified Workflow**: With RAGchain, you don’t need to manually handle the separate steps of retrieval and generation; the framework takes care of the entire process, making it easier to implement and maintain.


In [18]:
# This is an output parser used to convert the raw output from the LLM into a string.
# It ensures the LLM’s output is formatted as readable text.
from langchain_core.output_parsers import StrOutputParser
# This is a utility that simply passes the input data through without any modifications.
# It’s used when you want to allow the data to flow through a pipeline step unchanged.
from langchain_core.runnables import RunnablePassthrough

# LLM chain:
# The prompt is sent to the LLM.
# The LLM generates a response.
# The generated response is passed to StrOutputParser for formatting.
llm_chain = prompt | llm | StrOutputParser()

# RAG chain:
# 1. `context`: contextual information retrieved from the knowledge base.
#     The value is a pipeline that first uses retriever to find relevant documents,
#     and then passes the retrieved documents through format_docs to format them for further use.
#     Additionally, the `format_sources` step is added to include the sources.
# 1.1. `retriever`: Retrieves relevant documents from the knowledge base based on the user's query.
# 1.2. `format_docs`: Formats the retrieved documents.
# 1.3. `format_sources`: Extracts the sources of the retrieved documents (e.g., URLs).
# 2. `question`: the user's input query, which is passed directly through RunnablePassthrough().
#     The RunnablePassthrough simply forwards the original question without any modification.
# 3. `llm_chain`: Takes these inputs (context + sources + question) and generates a response using the LLM.
rag_chain = (
    {
        "context": retriever | format_docs,    # Retrieval Step for context
        "sources": retriever | format_sources, # Retrieve and format sources
        "question": RunnablePassthrough()      # Prompt Generation
    }
    | llm_chain                                # Generation Step
)

# 8 | Showcase

---
外部知識（教科書の第2章から）に関する以下の文脈について、RAGシステムに問い合わせを試みました：

> エーコは、３種類の記号負荷体を列挙している。その区分は、部分的に材料の種類に関係することは注目される：
> '同じタイプの、単数または複数のトークン（複製）がある記号'（例えば、印刷された単語、または同じ色の全く同じモデルの自動車）；
> 'そのトークンがあるタイプから製造されたとしても、材質的には独自性を持った記号'（例えば、ある人が話したまたは手書きされた言葉）；
> 'そのトークンがそのタイプである、またはタイプとトークンが同一の記号'（例えば、唯一の原画である油絵またはダイアナ王女の結婚衣裳)。 (Eco 1976, 178ff)

以下で、ベクターデータベースなしでは、LLMの回答が望ましくないものであることが観察できます。対照的に、RAGチェーンによって生成された回答は、より正確で文脈的に適切です。

私の質問は以下の通りです：

> トークンとタイプに関連して、エーコは３種類の記号負荷体を提示している。その区分について教えていただけませんか。

I attempted to query the RAG system regarding the following context (the external knowledge), which is from the second chapter of the textbook:

> エーコは、３種類の記号負荷体を列挙している。その区分は、部分的に材料の種類に関係することは注目される：
> ‘同じタイプの、単数または複数のトークン（複製）がある記号’（例えば、印刷された単語、または同じ色の全く同じモデルの自動車）；
> ‘そのトークンがあるタイプから製造されたとしても、材質的には独自性を持った記号’（例えば、ある人が話したまたは手書きされた言葉）；
> ‘そのトークンがそのタイプである、またはタイプとトークンが同一の記号’（例えば、唯一の原画である油絵またはダイアナ王女の結婚衣裳)。 (Eco 1976, 178ff)

Below, we can observe that without the vector database, the LLM's answers are less desirable. In contrast, answers generated by the RAG chain are more accurate and contextually appropriate.

My question is

> トークンとタイプに関連して、エーコは３種類の記号負荷体を提示している。その区分について教えていただけませんか。

In [19]:
# My question
query = "トークンとタイプに関連して、エーコは３種類の記号負荷体を提示している。その区分について教えていただけませんか。"

## 8.1 | Domain-specific answering (RAG)

In [20]:
# answer with the context from our vector DB
anwser = rag_chain.invoke(query)
display_answer(anwser)

エーコは、以下の**3種類の記号負荷体**を提示しています：

1. **同じタイプの、単数または複数のトークン**（複製）：
   - 例としては、印刷された単語や全く同じモデルの自動車が挙げられます。

2. **トークンがあるタイプから製造されたが、材質的には独自性を持った記号**：
   - 例えば、ある人が話したり手書きした言葉です。

3. **トークンがそのタイプである、またはタイプとトークンが同一の記号**：
   - 例には、唯一の原画である油絵やダイアナ王女の結婚衣裳が含まれます。

これらの区分は、記号の材料の種類にも関係しています。 

Thanks for asking!

## 8.2 | Domain-general answering

In [21]:
# answer without external knowledge
answer_without_knowledge = llm_chain.invoke({"context":"", "question": query})
display_answer(answer_without_knowledge)

エーコは、記号負荷体を以下の3種類に分類しています：

1. **トークン**：具体的な記号の実体で、個別の事例や現象を指します。
2. **タイプ**：トークンが属する一般的なカテゴリやクラスで、トークンがその特性を持つことを示します。
3. **記号体系**：トークンとタイプの関係を含む、より広範な構造やルールで構成される体系です。

これにより、記号の多様な意味や使用法を理解する手助けとなります。

Thanks for asking!

## 8.3 | Extracting sources

---
次のブロックは、関連文書を取得し、そのソースを抽出し、生成された回答とソースを別々に表示する方法を示しています。これは、RAGシステムを使用する際に透明性とコンテキストを提供するのに便利です。

The following block is to show how to retrieve relevant documents, extract their sources, and then display both the generated answer and the sources separately. This can be useful for providing transparency and context when using a RAG system.


In [24]:
# Separately extract the sources for independent use
retrieved_docs = retriever.get_relevant_documents(query)
sources = format_sources(retrieved_docs)

# Display the sources separately
print("Source metainfo:\n", "".join(sources))

Source:
 [Document(id='f9814612-c2ef-412e-8cdd-3a570497c6e4', metadata={'source': 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem02_japanese.html'}, page_content='記号負荷体のもう一つの区分は、パースによって導かれたトークンとタイプという言語的な概念と関連している (Peirce 1931-58, 4.537)。話や書かれたテクストの中の単語を例とすると、トークンの個数は（タイプに関係なく）単語の全数であり、タイプの個数は異なる単語の、繰り返しは無視した個数である。意味論の言葉では、トークンはそのタイプを例示する（タイプの具体例である）。‘Word’と‘word’は、同じタイプの具体例である。言語はトークンとタイプの間の区分、つまり特定の例と一般的な分類に依存する。これは分類の基礎である。John Lyonsは、あるものがあるタイプのトークンとして数えられるかどうかは、その人の目的によると述べている。例えば：\n\n\nたまたま同じように綴られまた発音されるが、意味は異なる単語はトークンに含めるべきか？\n大文字は、対応する小文字と同じタイプの具体例か？ \nイタリック体で印刷された単語は、ローマ字体の同じ単語は同じタイプの具体例か？\nXと言う人によって書かれた単語とYという人によって書かれた単語は同じか？  \n(Lyons 1977, 13-15)'), Document(id='4c24e834-717d-44ce-8554-f6dc2ad6e941', metadata={'source': 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem02_japanese.html'}, page_content='記号論的視点からは、次のようなことによってのみ、そのような疑問に回答できる。特定の何かを意味する行為という文脈で、異なる形式が、関係する記号使用者にとって重要ななにかを意味しているかどうかを考慮する。\n\n\u3000エーコは、３種類の記号負荷体を列挙している。その区分は、部分

In [23]:
# Display the sources separately
retrieved_docs

[Document(id='f9814612-c2ef-412e-8cdd-3a570497c6e4', metadata={'source': 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem02_japanese.html'}, page_content='記号負荷体のもう一つの区分は、パースによって導かれたトークンとタイプという言語的な概念と関連している (Peirce 1931-58, 4.537)。話や書かれたテクストの中の単語を例とすると、トークンの個数は（タイプに関係なく）単語の全数であり、タイプの個数は異なる単語の、繰り返しは無視した個数である。意味論の言葉では、トークンはそのタイプを例示する（タイプの具体例である）。‘Word’と‘word’は、同じタイプの具体例である。言語はトークンとタイプの間の区分、つまり特定の例と一般的な分類に依存する。これは分類の基礎である。John Lyonsは、あるものがあるタイプのトークンとして数えられるかどうかは、その人の目的によると述べている。例えば：\n\n\nたまたま同じように綴られまた発音されるが、意味は異なる単語はトークンに含めるべきか？\n大文字は、対応する小文字と同じタイプの具体例か？ \nイタリック体で印刷された単語は、ローマ字体の同じ単語は同じタイプの具体例か？\nXと言う人によって書かれた単語とYという人によって書かれた単語は同じか？  \n(Lyons 1977, 13-15)'),
 Document(id='4c24e834-717d-44ce-8554-f6dc2ad6e941', metadata={'source': 'http://www.visual-memory.co.uk/daniel/Documents/S4B/japanese/sem02_japanese.html'}, page_content='記号論的視点からは、次のようなことによってのみ、そのような疑問に回答できる。特定の何かを意味する行為という文脈で、異なる形式が、関係する記号使用者にとって重要ななにかを意味しているかどうかを考慮する。\n\n\u3000エーコは、３種類の記号負荷体を列挙している。その区分は、部分的に材料の種類に

# References

- https://github.com/huggingface/cookbook/tree/main/notebooks
- https://python.langchain.com/v0.2/docs/tutorials/rag/#retrieval-and-generation
- [生成AIの新展開！？―学術研究支援用ボットを作ってみた。その(1)](https://digitalnagasaki.hatenablog.com/entry/2024/10/22/101846)